### Simple two-output model

In this exercise, you will use the tournament data to build one model that makes two predictions: the scores of both teams in a given game. Your inputs will be the seed difference of the two teams, as well as the predicted score difference from the model you built in chapter 3.

The output from your model will be the predicted score for team 1 as well as team 2. This is called "multiple target regression": one model making more than one prediction.

In [6]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
import tensorflow as tf
import pandas as pd
import numpy as np

In [8]:
from sklearn.model_selection import train_test_split

In [14]:
games_tourney = pd.read_csv('../basketball_data/games_season_enriched.csv')

In [21]:
games_tourney_train, games_tourney_test = train_test_split(games_tourney, train_size=0.7,\
                                                          random_state=123)

In [22]:
games_tourney.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won,pred
0,1985,288,73,0,-3,-9,41,50,0,-3.601452
1,1985,5929,73,0,4,6,61,55,1,0.474164
2,1985,9884,73,0,5,-4,59,63,0,-0.414316
3,1985,73,288,0,3,9,50,41,1,3.601452
4,1985,3920,410,0,1,-9,54,63,0,8.176179


In [23]:
# Define the input
input_tensor = Input(shape=(2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(loss='mean_absolute_error', optimizer='adam')

In [24]:
# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=True,
  		  epochs=100,
  		  batch_size=16384)

Epoch 1/100
1/1 [==============================] - 0s 200ms/step - loss: 71.1538
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 71.1527
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 71.1516
Epoch 4/100
1/1 [==============================] - 0s 6ms/step - loss: 71.1505
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 71.1494
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 71.1483
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: 71.1472
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 71.1461
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 71.1450
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 71.1439
Epoch 11/100
1/1 [==============================] - 0s 13ms/step - loss: 71.1428
Epoch 12/100
1/1 [==============================] - 0s 7ms/step - loss: 71.1417
Epoch 13/100
1/1 [===========================

In [25]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[-0.6723117 , -0.25350887],
       [-0.48401794, -0.99180037]], dtype=float32), array([0.10000434, 0.09999995], dtype=float32)]
season        2001.171110
team_1        5592.724603
team_2        5638.273034
home             0.000000
seed_diff        0.023625
score_diff      -0.104961
score_1         71.032400
score_2         71.137361
won              0.501856
pred             0.064775
dtype: float64


In [27]:
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']],
               games_tourney_test[['score_1', 'score_2']], verbose=False))

71.00264739990234


### Classification and regression in one model

Now you will create a different kind of 2-output model. This time, you will predict the score difference, instead of both team's scores and then you will predict the probability that team 1 won the game. This is a pretty cool model: it is going to do both classification and regression!

In this model, turn off the bias, or intercept for each layer. Your inputs (seed difference and predicted score difference) have a mean of very close to zero, and your outputs both have means that are close to zero, so your model shouldn't need the bias term to fit the data well.

* Instructions

    * Create a single input layer with 2 columns.
    * The first output layer should have 1 unit with 'linear' activation and no bias term.
    * The second output layer should have 1 unit with 'sigmoid' activation and no bias term. Also, use the first output layer as an input to this layer.
    * Create a model with these input and outputs.



In [28]:
# Create an input layer with 2 columns
input_tensor = Input(shape=(2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

### Compile and fit the model

Now that you have a model with 2 outputs, compile it with 2 loss functions: mean absolute error (MAE) for 'score_diff' and binary cross-entropy (also known as logloss) for 'won'. Then fit the model with 'seed_diff' and 'pred' as inputs. For outputs, predict 'score_diff' and 'won'.

This model can use the scores of the games to make sure that close games (small score diff) have lower win probabilities than blowouts (large score diff).

The regression problem is easier than the classification problem because MAE punishes the model less for a loss due to random chance. For example, if score_diff is -1 and won is 0, that means team_1 had some bad luck and lost by a single free throw. The data for the easy problem helps the model find a solution to the hard problem.
* Instructions

    * Import Adam from keras.optimizers.
    * Compile the model with 2 losses: 'mean_absolute_error' and 'binary_crossentropy', and use the Adam optimizer with a learning rate of 0.01.
    * Fit the model with 'seed_diff' and 'pred' columns as the inputs and 'score_diff' and 'won' columns as the targets.
    * Use 10 epochs and a batch size of 16384.


In [30]:
# Import the Adam optimizer
from tensorflow.keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

Epoch 1/10
1/1 [==============================] - 0s 424ms/step - loss: 32.9832 - dense_6_loss: 19.8141 - dense_7_loss: 13.1691
Epoch 2/10
1/1 [==============================] - 0s 4ms/step - loss: 32.5773 - dense_6_loss: 19.6816 - dense_7_loss: 12.8957
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 32.1742 - dense_6_loss: 19.5493 - dense_7_loss: 12.6250
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 31.7740 - dense_6_loss: 19.4171 - dense_7_loss: 12.3569
Epoch 5/10
1/1 [==============================] - 0s 5ms/step - loss: 31.3768 - dense_6_loss: 19.2851 - dense_7_loss: 12.0916
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 30.9826 - dense_6_loss: 19.1536 - dense_7_loss: 11.8291
Epoch 7/10
1/1 [==============================] - 0s 6ms/step - loss: 30.5916 - dense_6_loss: 19.0223 - dense_7_loss: 11.5693
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 30.2039 - dense_6_loss: 18.8915 - dense_7_los

In [31]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(games_tourney_train.mean())

[array([[ 0.16504034],
       [-1.0573688 ]], dtype=float32), array([[1.4228725]], dtype=float32)]
season        2001.171110
team_1        5592.724603
team_2        5638.273034
home             0.000000
seed_diff        0.023625
score_diff      -0.104961
score_1         71.032400
score_2         71.137361
won              0.501856
pred             0.064775
dtype: float64


In [32]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability of a predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability of a predicted blowout game
print(sigmoid(10 * weight))

0.5349429451582145
0.8021838885585818


In [33]:
# Evaluate the model on new data
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']],
               [games_tourney_test[['score_diff']], games_tourney_test[['won']]], verbose=False))

[29.483139038085938, 18.926162719726562, 10.556974411010742]
